In this notebook we will perfrom two step predection  1st by identfy normal or abnormal then another round of predection the exact event type

# 1. Importing Libraries

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import sys
import os

%matplotlib inline

In [3]:
import pandas as pd
import numpy as np

# 2. Loading the Datasets


In [4]:
normal = pd.read_csv('real_normal.csv')
abnormal = pd.read_csv('real_abnormal.csv')

In [5]:
normal_df = normal
abnormal_df = abnormal

In [6]:
# Calculate the number of missing values in each dataset
normal_df_missing = normal_df.isnull().sum().to_frame('Normal')
abnormal_df_missing = abnormal_df.isnull().sum().to_frame('Abnomral')

# Combine the missing values data
missing_values = normal_df_missing.join(abnormal_df_missing, how='outer')

# Reset index to have 'Feature' column
missing_values = missing_values.rename_axis('Feature').reset_index()

# Display the missing values comparison
print(missing_values)


          Feature    Normal  Abnomral
0       timestamp         0         0
1           label         0         0
2            well         0         0
3              id         0         0
4       ABER-CKGL  10736697  13644040
5        ABER-CKP  10593466   9952326
6     ESTADO-DHSV   6307285  12699568
7       ESTADO-M1   6164054   8934991
8       ESTADO-M2   6164054   9019914
9      ESTADO-PXO   5606911   9089136
10  ESTADO-SDV-GL   7563940   8203039
11   ESTADO-SDV-P   5736080   2820063
12      ESTADO-W1   6164054   7770160
13      ESTADO-W2   5607326   8971462
14      ESTADO-XO   5606911   8726609
15       P-ANULAR   1382513   6029222
16       P-JUS-BS  12158183  20714067
17     P-JUS-CKGL   5079440   2742431
18      P-JUS-CKP  12014952  17408397
19     P-MON-CKGL  12158183  20405341
20      P-MON-CKP   5607366   1472485
21    P-MON-SDV-P  12158183  20714067
22          P-PDG   2488921   5427295
23           PT-P  12158183  20714067
24          P-TPT   1217710   4164073
25          

### 3.4.1 Missing Values %%


In [7]:
# Function to calculate missing value percentages
def missing_percentage(df, dataset_name):
    total = df.shape[0]
    missing_count = df.isnull().sum()
    missing_percent = (missing_count / total) * 100
    return missing_percent.to_frame(dataset_name)

# Calculate missing percentages for each dataset
normal_df_missing = missing_percentage(normal_df, 'Normal')
abnormal_df_missing = missing_percentage(abnormal_df, 'Abnormal')

# Combine the percentages
missing_percentages = normal_df_missing.join(abnormal_df_missing)
missing_percentages = missing_percentages.rename_axis('Feature').reset_index()

# Display the missing percentages
print(missing_percentages)

#features_to_drop = ['P-JUS-BS', 'P-JUS-CKP', 'P-MON-CKGL', 'T-MON-CKP', 'P-MON-SDV-P', 'PT-P', 'QBS', 'ABER-CKGL', ' ABER-CKP']


          Feature      Normal    Abnormal
0       timestamp    0.000000    0.000000
1           label    0.000000    0.000000
2            well    0.000000    0.000000
3              id    0.000000    0.000000
4       ABER-CKGL   88.308401   65.868475
5        ABER-CKP   87.130338   48.046219
6     ESTADO-DHSV   51.876872   61.308907
7       ESTADO-M1   50.698809   43.134895
8       ESTADO-M2   50.698809   43.544872
9      ESTADO-PXO   46.116356   43.879051
10  ESTADO-SDV-GL   62.212750   39.601296
11   ESTADO-SDV-P   47.178760   13.614241
12      ESTADO-W1   50.698809   37.511513
13      ESTADO-W2   46.119770   43.310964
14      ESTADO-XO   46.116356   42.128902
15       P-ANULAR   11.371049   29.106896
16       P-JUS-BS  100.000000  100.000000
17     P-JUS-CKGL   41.777953   13.239462
18      P-JUS-CKP   98.821937   84.041425
19     P-MON-CKGL  100.000000   98.509583
20      P-MON-CKP   46.120099    7.108623
21    P-MON-SDV-P  100.000000  100.000000
22          P-PDG   20.471159   26

## 2. Pre-processing real_instances

#### 2.1.  Drop features with >75^% of meissing values

In [7]:
# Drop features with high missing data (> 85%)
features_to_drop = ['P-JUS-BS', 'P-JUS-CKP', 'P-MON-CKGL', 'T-MON-CKP', 'P-MON-SDV-P', 'PT-P', 'QBS', 'ABER-CKGL', 'ABER-CKP']

normal.drop(columns=features_to_drop, inplace=True)
abnormal.drop(columns=features_to_drop, inplace=True)


#### 2.3.  Impute rest of missing features 

In [8]:
# Function to impute missing values using forward and backward filling within each 'class' and 'well' group
def impute_class_well_fill(df):
    # Identify columns that contain missing values
    columns_with_missing_values = df.columns[df.isnull().any()].tolist()
    
    # Iterate over each column with missing values
    for column in columns_with_missing_values:
        # Group by both 'class' and 'well' and apply forward fill, then backward fill
        df[column] = df.groupby(['label'])[column].transform(lambda x: x.ffill().bfill())
    
    return df


# Impute missing values in all columns with missing values
x_normal = impute_class_well_fill(normal)

# Check if missing values are filled
print(x_normal.isnull().sum())


timestamp        0
label            0
well             0
id               0
ESTADO-DHSV      0
ESTADO-M1        0
ESTADO-M2        0
ESTADO-PXO       0
ESTADO-SDV-GL    0
ESTADO-SDV-P     0
ESTADO-W1        0
ESTADO-W2        0
ESTADO-XO        0
P-ANULAR         0
P-JUS-CKGL       0
P-MON-CKP        0
P-PDG            0
P-TPT            0
QGL              0
T-JUS-CKP        0
T-PDG            0
T-TPT            0
class            0
state            0
dtype: int64


In [9]:

# Impute missing values in all columns with missing values
x_abnormal = impute_class_well_fill(abnormal)

# Check if missing values are filled
print(x_abnormal.isnull().sum())

timestamp              0
label                  0
well                   0
id                     0
ESTADO-DHSV            0
ESTADO-M1              0
ESTADO-M2              0
ESTADO-PXO             0
ESTADO-SDV-GL      77477
ESTADO-SDV-P           0
ESTADO-W1              0
ESTADO-W2              0
ESTADO-XO              0
P-ANULAR               0
P-JUS-CKGL         77477
P-MON-CKP              0
P-PDG                  0
P-TPT                  0
QGL                77477
T-JUS-CKP        4809035
T-PDG                  0
T-TPT                  0
class                  0
state                  0
dtype: int64


In [11]:
missing_info = {}

# Columns with remaining missing values after filling
columns_with_missing = ['ESTADO-SDV-GL', 'P-JUS-CKGL', 'QGL', 'T-JUS-CKP']

# Loop through each column with missing values to check associated 'class' and 'well'
for column in columns_with_missing:
    # Get the rows where the column still has missing values
    missing_rows = x_abnormal[x_abnormal[column].isna()]
    # Get the unique classes and wells associated with missing values in this column
    missing_info[column] = {
        'label': missing_rows['label'].unique(),
        'well': missing_rows['well'].unique(),
        'count': missing_rows.shape[0]
    }

# Display the results
for column, info in missing_info.items():
    print(f"Column '{column}' has {info['count']} missing values.")
    print(f"Associated Classes: {info['label']}")
    print(f"Associated Wells: {info['well']}")
    print("-" * 40)

Column 'ESTADO-SDV-GL' has 77477 missing values.
Associated Classes: [6]
Associated Wells: ['WELL-00004' 'WELL-00002']
----------------------------------------
Column 'P-JUS-CKGL' has 77477 missing values.
Associated Classes: [6]
Associated Wells: ['WELL-00004' 'WELL-00002']
----------------------------------------
Column 'QGL' has 77477 missing values.
Associated Classes: [6]
Associated Wells: ['WELL-00004' 'WELL-00002']
----------------------------------------
Column 'T-JUS-CKP' has 4809035 missing values.
Associated Classes: [8]
Associated Wells: ['WELL-00026' 'WELL-00019' 'WELL-00030' 'WELL-00027' 'WELL-00029'
 'WELL-00031' 'WELL-00032' 'WELL-00028' 'WELL-00025']
----------------------------------------


In [10]:
# List of wells to delete with label 6
wells_to_delete = ['WELL-00026', 'WELL-00019', 'WELL-00030', 'WELL-00027', 
                   'WELL-00029', 'WELL-00031', 'WELL-00032', 'WELL-00028', 'WELL-00025']

# Filter out rows where label is 6 and well is in the specified list
x_abnormal = x_abnormal[~((x_abnormal['label'] == 6) | (x_abnormal['well'].isin(wells_to_delete)))]
x_normal = x_normal[~((x_normal['label'] == 6) | (x_normal['well'].isin(wells_to_delete)))]

# Display the DataFrame to confirm the rows have been deleted


In [13]:
print(x_abnormal.isnull().sum())

timestamp        0
label            0
well             0
id               0
ESTADO-DHSV      0
ESTADO-M1        0
ESTADO-M2        0
ESTADO-PXO       0
ESTADO-SDV-GL    0
ESTADO-SDV-P     0
ESTADO-W1        0
ESTADO-W2        0
ESTADO-XO        0
P-ANULAR         0
P-JUS-CKGL       0
P-MON-CKP        0
P-PDG            0
P-TPT            0
QGL              0
T-JUS-CKP        0
T-PDG            0
T-TPT            0
class            0
state            0
dtype: int64


In [11]:
x_abnormal['is_normal'] = 0
x_normal['is_normal'] = 1

C:\Users\ma123\AppData\Local\Temp\ipykernel_18816\1904061805.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_abnormal['is_normal'] = 0
C:\Users\ma123\AppData\Local\Temp\ipykernel_18816\1904061805.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_normal['is_normal'] = 1


In [15]:
x_abnormal

,timestamp,label,well,id,ESTADO-DHSV,ESTADO-M1,ESTADO-M2,ESTADO-PXO,ESTADO-SDV-GL,ESTADO-SDV-P,...,P-MON-CKP,P-PDG,P-TPT,QGL,T-JUS-CKP,T-PDG,T-TPT,class,state,is_normal
0,2014-12-17 14:27:45,9,WELL-00042,20141217142745,1.0,1.0,1.0,0.0,1.0,1.0,...,2190777.0,18979920.0,10180070.0,3.032789,40.20570,74.58371,58.95389,0.0,0.0,0
1,2014-12-17 14:27:46,9,WELL-00042,20141217142745,1.0,1.0,1.0,0.0,1.0,1.0,...,2190779.0,18979930.0,10180060.0,3.033681,40.20566,74.58370,58.95395,0.0,0.0,0
2,2014-12-17 14:27:47,9,WELL-00042,20141217142745,1.0,1.0,1.0,0.0,1.0,1.0,...,2190780.0,18979940.0,10180050.0,3.034572,40.20562,74.58369,58.95401,0.0,0.0,0
3,2014-12-17 14:27:48,9,WELL-00042,20141217142745,1.0,1.0,1.0,0.0,1.0,1.0,...,2190781.0,18979950.0,10180040.0,3.035463,40.20559,74.58368,58.95407,0.0,0.0,0
4,2014-12-17 14:27:49,9,WELL-00042,20141217142745,1.0,1.0,1.0,0.0,1.0,1.0,...,2190782.0,18979960.0,10180040.0,3.036354,40.20555,74.58368,58.95413,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20714062,2014-03-20 04:20:34,5,WELL-00020,20140319110000,1.0,1.0,0.0,0.0,0.0,1.0,...,1311047.0,32308910.0,4631294.0,0.000000,31.23388,118.15850,87.30427,5.0,0.0,0
20714063,2014-03-20 04:20:35,5,WELL-00020,20140319110000,1.0,1.0,0.0,0.0,0.0,1.0,...,1310748.0,32308910.0,4630961.0,0.000000,31.23269,118.15850,87.30622,5.0,0.0,0
20714064,2014-03-20 04:20:36,5,WELL-00020,20140319110000,1.0,1.0,0.0,0.0,0.0,1.0,...,1310449.0,32308910.0,4630629.0,0.000000,31.23150,118.15850,87.30817,5.0,0.0,0
20714065,2014-03-20 04:20:37,5,WELL-00020,20140319110000,1.0,1.0,0.0,0.0,0.0,1.0,...,1310150.0,32308910.0,4630296.0,0.000000,31.23031,118.15850,87.31013,5.0,0.0,0


### 2.1.  resampling

In [12]:
from sklearn.model_selection import train_test_split
x_normal_sample, _ = train_test_split(x_normal, test_size=0.80)


In [12]:
print(x_abnormal.shape)

print(x_abnormal['label'].value_counts(normalize=True))  # Check class proportions


(15827555, 25)
7    0.496915
4    0.233118
9    0.166505
3    0.043238
5    0.027762
2    0.017501
1    0.014961
Name: label, dtype: float64


In [13]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Separate features and labels
X = x_abnormal.select_dtypes(include=[float, int])  # Keep only numeric columns
y = x_abnormal['label']  # Define the label column

# Initialize SMOTE for oversampling only minority classes
smote = SMOTE(sampling_strategy='minority', random_state=42)

# Apply SMOTE to create a balanced dataset
X_resampled, y_resampled = smote.fit_resample(X, y)

# Check the distribution of classes after applying SMOTE
print("Original class distribution:", Counter(y))
print("Resampled class distribution:", Counter(y_resampled))

# Combine resampled features and labels back into a DataFrame if needed
x_abnormal_resampled = X_resampled.copy()
x_abnormal_resampled['label'] = y_resampled


Original class distribution: Counter({7: 7864945, 4: 3689683, 9: 2635372, 3: 684352, 5: 439408, 2: 277001, 1: 236794})
Resampled class distribution: Counter({7: 7864945, 1: 7864945, 4: 3689683, 9: 2635372, 3: 684352, 5: 439408, 2: 277001})


In [14]:
print(x_abnormal_resampled.shape)

print(x_abnormal_resampled['label'].value_counts(normalize=True))  # Check class proportions


(23455706, 23)
7    0.335311
1    0.335311
4    0.157304
9    0.112355
3    0.029176
5    0.018734
2    0.011810
Name: label, dtype: float64


In [14]:
merged_df = pd.concat([x_normal_sample, x_abnormal_resampled], ignore_index=True)


In [16]:
print(merged_df.shape)

print(merged_df['label'].value_counts(normalize=True))  # Check class proportions


(25858696, 25)
7    0.304151
1    0.304151
4    0.142686
9    0.101914
0    0.092928
3    0.026465
5    0.016993
2    0.010712
Name: label, dtype: float64


## 3 ML Classifcation Model


### 3.1 Expriment 1: XGBoost Classfication

Model Loading & Prepration

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming 'r_df_sample' is your dataset and 'class' is the target variable
# X will be the features, y will be the target class
X = merged_df.drop(columns=['is_normal'])  # Features (drop the class column)
y = merged_df['is_normal']  # Target (the class labels)

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)


In [16]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform y for both training and test sets
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Check the transformed class labels
print(np.unique(y_train))  # Ensure they are integers starting from 0


[0 1]


In [17]:
# Convert class labels to integers if needed
y_train = y_train.astype(int)
y_test = y_test.astype(int)


In [18]:
# Columns to drop from the dataset
columns_to_drop = ['timestamp', 'class', 'well', 'id','state','label']

# Drop these columns from both the training and test sets
X_train = X_train.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)

# Verify that the columns have been dropped
print("Training set columns after dropping:", X_train.columns)
print("Test set columns after dropping:", X_test.columns)


Training set columns after dropping: Index(['ESTADO-DHSV', 'ESTADO-M1', 'ESTADO-M2', 'ESTADO-PXO', 'ESTADO-SDV-GL',
       'ESTADO-SDV-P', 'ESTADO-W1', 'ESTADO-W2', 'ESTADO-XO', 'P-ANULAR',
       'P-JUS-CKGL', 'P-MON-CKP', 'P-PDG', 'P-TPT', 'QGL', 'T-JUS-CKP',
       'T-PDG', 'T-TPT'],
      dtype='object')
Test set columns after dropping: Index(['ESTADO-DHSV', 'ESTADO-M1', 'ESTADO-M2', 'ESTADO-PXO', 'ESTADO-SDV-GL',
       'ESTADO-SDV-P', 'ESTADO-W1', 'ESTADO-W2', 'ESTADO-XO', 'P-ANULAR',
       'P-JUS-CKGL', 'P-MON-CKP', 'P-PDG', 'P-TPT', 'QGL', 'T-JUS-CKP',
       'T-PDG', 'T-TPT'],
      dtype='object')


In [19]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Model Training & Test

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize Logistic Regression model for binary classification
logistic_model = LogisticRegression(solver='saga', max_iter=1000,random_state=42)

# Train the model on the scaled data
logistic_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = logistic_model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 97.42%


In [20]:
# X will be the features, y will be the target class
X = x_abnormal_resampled.drop(columns=['label'])  # Features (drop the class column)
y = x_abnormal_resampled['label']  # Target (the class labels)

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)


# Fit and transform y for both training and test sets
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Check the transformed class labels
print(np.unique(y_train))  # Ensure they are integers starting from 0


# Convert class labels to integers if needed
y_train = y_train.astype(int)
y_test = y_test.astype(int)





[0 1 2 3 4 5 6]


In [23]:
# Columns to drop from the dataset
columns_to_drop = [ 'class', 'id','state','is_normal']

# Drop these columns from both the training and test sets
X_train = X_train.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)

# Verify that the columns have been dropped
print("Training set columns after dropping:", X_train.columns)
print("Test set columns after dropping:", X_test.columns)


# Fit the scaler on the training data and transform both training and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Training set columns after dropping: Index(['ESTADO-DHSV', 'ESTADO-M1', 'ESTADO-M2', 'ESTADO-PXO', 'ESTADO-SDV-GL',
       'ESTADO-SDV-P', 'ESTADO-W1', 'ESTADO-W2', 'ESTADO-XO', 'P-ANULAR',
       'P-JUS-CKGL', 'P-MON-CKP', 'P-PDG', 'P-TPT', 'QGL', 'T-JUS-CKP',
       'T-PDG', 'T-TPT'],
      dtype='object')
Test set columns after dropping: Index(['ESTADO-DHSV', 'ESTADO-M1', 'ESTADO-M2', 'ESTADO-PXO', 'ESTADO-SDV-GL',
       'ESTADO-SDV-P', 'ESTADO-W1', 'ESTADO-W2', 'ESTADO-XO', 'P-ANULAR',
       'P-JUS-CKGL', 'P-MON-CKP', 'P-PDG', 'P-TPT', 'QGL', 'T-JUS-CKP',
       'T-PDG', 'T-TPT'],
      dtype='object')


In [24]:
# Initialize XGBoost classifier for multi-class classification
xgb_simple_model = XGBClassifier(objective='multi:softmax', num_class=len(np.unique(y_train)), random_state=42)

# Train the model on the scaled data
xgb_simple_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = xgb_simple_model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
 

Model Accuracy: 99.98%
